In [37]:
import numpy as np
import pandas as pd


lex = pd.read_csv('Lexique380.utf8.csv')
#lex = pd.read_csv('test.csv')
lex = lex[(lex.ortho.str.contains('-| ') == False) & (lex.phon.str.contains('°') == False)]  # suppress schwa
lex = lex.drop_duplicates(subset='ortho', keep="first")
lex = lex[['ortho','phon', 'p_cvcv','nbhomogr','cv-cv','syll']]
dic = lex.set_index('ortho').to_dict()

p_dic = dic['p_cvcv']
n_dic = dic['nbhomogr']
cv_dic = dic['cv-cv']
p_cv_dic = dic['syll']
phon_dic = dic['phon']

lex['cv-div'] = lex['cv-cv'].apply(lambda x: x.split('-'))

In [3]:
##### Creating the possible configurations for 1 spoken syllable

In [4]:
lex['cv-div'] = lex['cv-cv'].apply(lambda x: x.split('-'))


flat_list = [item for sublist in lex['cv-div'] for item in sublist]
flat_set = set(flat_list)
flatset_l = list(flat_set)



lpc_syl_config = pd.DataFrame(flatset_l,columns =['spoken_config'])


#lpc_syl_config.to_csv('lpc_syl_config.csv')



In [18]:
# number of gestures of each syll

dev_syl = pd.read_csv('lpc_syl_configurations.csv')
dev_syl['lpc_n'] = dev_syl['LPC_config'].apply(lambda x: x.split('-'))
dev_syl['lpc_n'] = dev_syl['lpc_n'].apply(lambda x: len(x))

dic2 = dev_syl.set_index('spoken_config').to_dict()
g_cv_dic = dic2['LPC_config']
syl_dic = dic2['lpc_n']

print(syl_dic)

{'CCVYC': 4, 'CYVC': 3, 'YV': 1, 'CCVC': 3, 'CCYVCC': 5, 'CCCYV': 4, 'CCCVC': 4, 'VCC': 3, 'VY': 2, 'CYVY': 3, 'V': 1, 'CCVCCC': 5, 'CCCV': 3, 'CVCCC': 4, 'CC': 1, 'CVYCCC': 5, 'CCYV': 3, 'CYVCC': 4, 'CCCYVC': 5, 'YVY': 2, 'CCC': 1, 'CYVCCC': 5, 'CCVY': 3, 'CCYVYC': 5, 'VYC': 3, 'CVCC': 3, 'YVC': 3, 'CCV': 2, 'CVY': 2, 'CVYC': 3, 'VC': 2, 'CCCVCCC': 1, 'CVCCCC': 5, 'YVCC': 3, 'CCVYCC': 5, 'C': 1, 'CCCVCC': 5, 'CCYVY': 4, 'CVC': 2, 'CCVCC': 4, 'VYCC': 4, 'VCCC': 4, 'CVYCC': 4, 'CCCVY': 4, 'CCYVC': 4, 'CYV': 2, 'CV': 1}


In [35]:
def get_LPC_cv(word):
    LPC_cv = ''    
    if word in cv_dic:
        cv_lst = cv_dic[word].split('-')
        for syl in cv_lst:
            LPC_cv = LPC_cv + g_cv_dic[syl] + '-'
        return LPC_cv[:-1]

    else: 
        return word

In [38]:
def get_LPC_p(word):
    lpc_cv = get_LPC_cv(word)
    new_word = ''
    phon = phon_dic[word]
    if lpc_cv == cv_dic[word]:
        return p_cv_dic[word]
    else:
        l_lpc = lpc_cv.split('-')
        for syl in l_lpc:
            new_word += phon[:len(syl)]+'-'
            phon = phon[len(syl):]
        return new_word[:-1]


In [51]:
def get_LPC_p_sentence(sentence):
    sentence = sentence.lower()
    replace_chars = [',','.']
    for replace_char in replace_chars:
        sentence = sentence.replace(replace_char,'')
        
    sentence = sentence.replace('-', ' ')
    snt_to_wlst = sentence.split(' ')
    new_sentence = ''
    no_in_db = []
    more_ortho ={}

    
    for word in snt_to_wlst:
        if word in cv_dic:
            phon_w = get_LPC_p(word)
            new_sentence += phon_w + '   '
            if n_dic[word] != 1:
                more_ortho[word] = phon_dic[word]
        else: 
            no_in_db.append(word)
        
    
    if no_in_db == [] and more_ortho == []:
        return new_sentence
    if no_in_db == [] and more_ortho != []:
        return new_sentence + f',check the cv for: {more_ortho}'
    else:
        return new_sentence + f',except {no_in_db}, check the phon for: {more_ortho}'

In [54]:
get_LPC_p_sentence('luisi Faites attention à ne pas toucher la poêle quand elle est chaude.')

"fE-t   a-t@-s-j§   a   n2   pa   tu-Se   la   p-wa-l   k@   E-l   e   So-d   ,except ['luisi'], check the phon for: {'faites': 'fEt', 'attention': 'at@sj§', 'pas': 'pa', 'toucher': 'tuSe', 'la': 'la', 'poêle': 'pwal', 'quand': 'k@', 'est': 'e', 'chaude': 'Sod'}"

In [6]:
def get_LPC_w(word):
    LPC_gustures = 0     
    if word in cv_dic:
        cv_lst = cv_dic[word].split('-')
        for syl in cv_lst:
            LPC_gustures += syl_dic[syl]
        return LPC_gustures

    else: 
        return word


In [8]:
def get_LPC_g(sentence):
    sentence = sentence.lower()
    replace_chars = [',','.']
    for replace_char in replace_chars:
        sentence = sentence.replace(replace_char,'')
        
    sentence = sentence.replace('-', ' ')
    snt_to_wlst = sentence.split(' ')
    LPC_gustures = 0 
    no_in_db = []
    more_ortho ={}

    
    for word in snt_to_wlst:
        if word in cv_dic:
            LPC_gustures += get_LPC_w(word)
            if n_dic[word] != 1:
                more_ortho[word] = get_LPC_w(word)
        else: 
            no_in_db.append(word)
        
    
    if no_in_db == [] and more_ortho == []:
        return f'{LPC_gustures}'
    if no_in_db == [] and more_ortho != []:
        return f'{LPC_gustures}, check the cv for: {more_ortho}'
    else:
        return f'{LPC_gustures}, except {no_in_db}, check the cv for: {more_ortho}'


    


In [18]:
def get_LPC_g1(sentence):
    sentence = sentence.lower()
    sentence = sentence.replace('.', '')
    sentence = sentence.replace('-', ' ')
    snt_to_wlst = sentence.split(" ")
    LPC_gustures = 0 
    e = 0
    no_in_db = []
    more_ortho ={}

    
    for word in snt_to_wlst:
        if word in p_dic:
            #print(word, p_dic[word])
            e = 1
            if p_dic[word][0] == 'V':
                LPC_gustures += 1
            for letter in p_dic[word]:
                if letter == 'C':
                    LPC_gustures += 1
            if n_dic[word] != 1:
                more_ortho[word] = p_dic[word]

        else: 
            e = 0
        if e == 0 :
            no_in_db.append(word)
        
    
    if no_in_db == [] and more_ortho == []:
        return f'{LPC_gustures}'
    if no_in_db == [] and more_ortho != []:
        return f'{LPC_gustures}, check the cv for: {more_ortho}'
    else:
        return f'{LPC_gustures}, except {no_in_db}, check the cv for: {more_ortho}'


In [9]:
def get_number_of_letters(sentence):
    sentence = sentence.lower()
    replace_chars = [',','.',' ']
    for replace_char in replace_chars:
        sentence = sentence.replace(replace_char,'')
    return len(sentence)
    

In [10]:
def get_number_of_words(sentence):
    sentence = sentence.replace('-',' ')
    w_lst = sentence.split(' ')
    return len(w_lst)
    

## Counting the numbers of LPC gestures for both sentences in to the file

In [53]:
df = pd.read_csv('CUSTIME_sentence_predictability.csv', encoding="utf-8")

for index,row in df.iterrows():
    p_sentence = row['Predictable end sentence']
    pred_end_n_LPC = get_LPC_g(p_sentence)
    pred_num_ltr = get_number_of_letters(p_sentence)
    pred_num_words = get_number_of_words(p_sentence)
    phon_pred = get_LPC_p_sentence(p_sentence)
    
    df.loc[index,"pred_num_letters"] = pred_num_ltr
    df.loc[index,"pred_end_n_LPC"] = pred_end_n_LPC
    df.loc[index,"pred_num_words"] = pred_num_words
    
    unp_sentence = row['Unpredictable end sentence']
    unpred_end_n_LPC = get_LPC_g(unp_sentence)
    unpred_num_ltr = get_number_of_letters(unp_sentence)
    unpred_num_words = get_number_of_words(unp_sentence)
    phon_unpred = get_LPC_p_sentence(unp_sentence)
    df.loc[index,"unpred_num_letters"] = unpred_num_ltr
    df.loc[index,"unpred_end_n_LPC"] = unpred_end_n_LPC
    df.loc[index,"unpred_num_words"] = unpred_num_words
    
    df["delta_tletters"] = df["pred_num_letters"] - df["unpred_num_letters"]
    df["delta_words"] = df["pred_num_words"] - df["unpred_num_words"]
    # Creating the LPC delta should be done on the csv after verifing manually
    
    df.loc[index,"phon_pred"] = phon_pred
    df.loc[index,"phon_unpred"] = phon_unpred
    

df.to_csv('CUSTIME_sentence_predictability_with_LPC.csv', encoding="utf-8-sig")



